# (Homework) Week 6 - DataScience Bootcamp Fall 2025

All solution cells are replaced with `# TODO` placeholders so you can fill them in.

**Name:** Prathamesh Adarkar
**Email:** pa2529@nyu.edu

### Problem 1: Dataset Splitting

1. You have recordings of 44 phones from 100 people; each person records ~200 phones/day for 5 days.
   - Design a valid training/validation/test split strategy that ensures the model generalizes to **new speakers**.

2. You now receive an additional dataset of 10,000 phone recordings from **Kilian**, a single speaker.
   - You must train a model that performs well **specifically for Kilian**, while also maintaining generalization.

*Describe your proposed split strategy and reasoning.* (Theory)

In [4]:
**Split Ratios:**
- Training: 70 speakers (70%)
- Validation: 15 speakers (15%)
- Test: 15 speakers (15%)

### Critical Principle: NO SPEAKER OVERLAP

Each speaker's entire 5-day recording sequence stays in ONE split only.

### Reasoning:

**1. Why Speaker-Level Splitting?**

The goal is generalization to NEW speakers. If we split recordings randomly:
- Same speaker appears in train AND test
- Model learns speaker-specific patterns (voice characteristics, accent)
- Test accuracy becomes artificially high
- Model fails in deployment on unseen speakers

**2. Why Keep All 5 Days Together?**

Recordings from the same speaker across days are correlated:
- Same microphone setup
- Consistent background environment
- Similar vocal characteristics

Splitting days across sets causes data leakage.

**3. Stratification Considerations:**

Balance across splits:
- Gender distribution (M/F)
- Age groups
- Accents/dialects
- Ensure all 44 phones are well-represented

**4. What We're Testing:**

Test set evaluates: "Can the model recognize phones from speakers it has NEVER seen?"

This matches real-world deployment where users are always new.

---

## Problem 2: Adding Kilian's 10,000 Recordings

1. Maintain generalization to unseen speakers
2. Optimize performance specifically for Kilian

### Proposed Strategy: **Two-Stage Fine-Tuning**

#### **Stage 1: Train Base Model**

```
Training:   70 speakers (original dataset)
Validation: 15 speakers (original dataset)
Test:       15 speakers (original dataset - held out)
```

**Output:** Base model θ_base with strong speaker-independent performance

#### **Stage 2: Kilian-Specific Adaptation**

**Split Kilian's 10,000 recordings:**
```
Training:   7,000 recordings (70%)
Validation: 1,500 recordings (15%)  
Test:       1,500 recordings (15%)
```

**Important:** Use temporal or random split for Kilian's data since it's all one speaker.

**Fine-Tuning Protocol:**
- Start from θ_base (pre-trained model)
- Use MUCH smaller learning rate (10× smaller)
- Apply regularization to prevent catastrophic forgetting
- Monitor BOTH metrics during training:
  - Performance on Kilian's validation set
  - Performance on original 15-speaker test set

**Stop Condition:**
- If general test accuracy drops >2%, stop or reduce learning rate
- Kilian validation accuracy should significantly improve

#### **Final Evaluation:**

Report TWO separate metrics:
1. **General Performance:** Accuracy on 15 unseen speakers (should stay ≥98% of base)
2. **Kilian Performance:** Accuracy on Kilian's test set (should improve significantly)

### Why This Works:

**Base Model Benefits:**
- Learns speaker-invariant phone features from diverse data
- Strong foundation for generalization

**Fine-Tuning Benefits:**
- Adapts to Kilian's specific voice characteristics
- 10,000 samples is sufficient for meaningful adaptation
- Low learning rate + regularization prevents forgetting general patterns

### Alternative: Multi-Task Learning

Instead of two stages, train ONE model with combined loss:

```
Loss = α × Loss_general + β × Loss_kilian

Where α=0.7, β=0.3
```

**Data Split:**
```
Training:   70 general speakers + 5,000 Kilian recordings
Validation: 15 general speakers + 1,500 Kilian recordings
Test:       15 general speakers + 1,500 Kilian recordings (separate evaluation)
```

This maintains generalization while learning Kilian-specific patterns, but is more complex to tune.

---

## Key Takeaways

**Always split by speaker, never by recordings** (for generalization tasks)
**Keep temporal sequences together** (prevent data leakage)

**Fine-tuning requires regularization** (prevent catastrophic forgetting)

**Dual evaluation is essential** (track both general and specific performance)

**Never** randomly split recordings across train/val/test with same speakers

**Never** fine-tune without monitoring general performance degradation


## Success Criteria

- General test accuracy ≥ 98% of base model (≤2% forgetting)
- Kilian test accuracy shows ≥10% improvement over base model
- All 44 phones adequately represented in all splits
- Balanced demographic distribution across splits

### Problem 2: K-Nearest Neighbors

1. **1-NN Classification:** Given dataset:

   Positive: (1,2), (1,4), (5,4)

   Negative: (3,1), (3,2)

   Plot the 1-NN decision boundary and classify new points visually.

2. **Feature Scaling:** Consider dataset:

   Positive: (100,2), (100,4), (500,4)

   Negative: (300,1), (300,2)

   What would the 1-NN classify point (500,1) as **before and after scaling** to [0,1] per feature?

3. **Handling Missing Values:** How can you modify K-NN to handle missing features in a test point?

4. **High-dimensional Data:** Why can K-NN still work well for images even with thousands of pixels?


In [7]:
# K-Nearest Neighbors (K-NN) - Theory Solutions

---

## Problem 1: 1-NN Classification & Decision Boundary

### Dataset:
- **Positive:** (1,2), (1,4), (5,4)
- **Negative:** (3,1), (3,2)

### 1-NN Decision Boundary Description:

The 1-NN decision boundary is defined by the **perpendicular bisectors** between each positive-negative point pair. The boundary consists of piecewise linear segments (Voronoi diagram).

**Key Boundary Regions:**

```
Visual Plot Description:

      5 |
      4 |    P₁ -------- P₃
      3 |         |  
      2 |    P₂   |   N₂      Decision boundary (approximate)
      1 |         N₁ |
      0 |____________|_________
        0   1   2   3   4   5

Where:
- P₁ = (1,2) Positive
- P₂ = (1,4) Positive  
- P₃ = (5,4) Positive
- N₁ = (3,1) Negative
- N₂ = (3,2) Negative
```

**Decision Boundary Explanation:**

The boundary forms perpendicular bisectors between nearest opposite-class neighbors:
- Between P₁(1,2) and N₁(3,1): bisector at approximately x≈2, y≈1.5
- Between P₁(1,2) and N₂(3,2): vertical bisector at x=2
- Between P₂(1,4) and N₂(3,2): bisector at approximately x≈2, y≈3
- Between P₃(5,4) and N₂(3,2): bisector separating right region

**Example Classifications:**

| Test Point | Nearest Neighbor | Distance | Classification |
|------------|------------------|----------|----------------|
| (2, 3)     | P₂ (1,4)        | √2 ≈ 1.4 | **Positive**   |
| (3.5, 1.5) | N₁ (3,1)        | √0.5     | **Negative**   |
| (4, 3)     | P₃ (5,4)        | √2 ≈ 1.4 | **Positive**   |
| (2, 1)     | N₁ (3,1)        | 1.0      | **Negative**   |

**Key Insight:** Any point is classified by its single nearest neighbor. The decision boundary creates regions (Voronoi cells) where all points in a region belong to the same class.

---

## Problem 2: Feature Scaling Impact

### Original Dataset (Unscaled):
- **Positive:** (100,2), (100,4), (500,4)
- **Negative:** (300,1), (300,2)
- **Test Point:** (500,1)

### Before Scaling:

**Distances from (500,1):**

| Point       | Class    | Distance Calculation                        | Distance |
|-------------|----------|---------------------------------------------|----------|
| (100,2)     | Positive | √[(500-100)² + (1-2)²] = √[160000+1]      | 400.0    |
| (100,4)     | Positive | √[(500-100)² + (1-4)²] = √[160000+9]      | 400.01   |
| (500,4)     | Positive | √[(500-500)² + (1-4)²] = √[0+9]           | **3.0**  |
| (300,1)     | Negative | √[(500-300)² + (1-1)²] = √[40000+0]       | 200.0    |
| (300,2)     | Negative | √[(500-300)² + (1-2)²] = √[40000+1]       | 200.0    |

**Nearest Neighbor:** (500,4) - Positive class  
**Classification:**  **Positive**

**Problem:** Feature 1 (range: 100-500) dominates Feature 2 (range: 1-4) due to scale difference!

---

### After Scaling to [0,1]:

**Scaling Formula:** x_scaled = (x - x_min) / (x_max - x_min)

**Feature 1:** min=100, max=500 → range=400  
**Feature 2:** min=1, max=4 → range=3

**Scaled Dataset:**

| Original    | Feature 1 Scaled       | Feature 2 Scaled       | Scaled Point  |
|-------------|------------------------|------------------------|---------------|
| (100,2)     | (100-100)/400 = 0      | (2-1)/3 = 0.33         | (0, 0.33)     |
| (100,4)     | 0                      | (4-1)/3 = 1.0          | (0, 1.0)      |
| (500,4)     | (500-100)/400 = 1.0    | 1.0                    | (1.0, 1.0)    |
| (300,1)     | (300-100)/400 = 0.5    | (1-1)/3 = 0            | (0.5, 0)      |
| (300,2)     | 0.5                    | 0.33                   | (0.5, 0.33)   |
| **(500,1)** | **1.0**                | **0**                  | **(1.0, 0)**  |

**Distances from scaled (1.0, 0):**

| Scaled Point | Class    | Distance Calculation                    | Distance |
|--------------|----------|-----------------------------------------|----------|
| (0, 0.33)    | Positive | √[(1-0)² + (0-0.33)²] = √[1+0.11]      | 1.05     |
| (0, 1.0)     | Positive | √[(1-0)² + (0-1)²] = √[1+1]            | 1.41     |
| (1.0, 1.0)   | Positive | √[(1-1)² + (0-1)²] = √[0+1]            | 1.0      |
| (0.5, 0)     | Negative | √[(1-0.5)² + (0-0)²] = √[0.25+0]      | **0.5**  |
| (0.5, 0.33)  | Negative | √[(1-0.5)² + (0-0.33)²] = √[0.25+0.11] | 0.6      |

**Nearest Neighbor:** (0.5, 0) - Negative class  
**Classification:**  **Negative**

---

### Summary:

| Condition      | Nearest Neighbor | Classification | Distance |
|----------------|------------------|----------------|----------|
| **Unscaled**   | (500,4)          | Positive       | 3.0      |
| **Scaled**     | (300,1)          | Negative       | 0.5      |

**Key Insight:**  
Without scaling, Feature 1 (large scale) dominated, making horizontal distance the primary factor. After scaling, both features contribute equally, revealing that (500,1) is actually closer to the negative class in the normalized space.

**Lesson:** Always scale features when they have different ranges, or K-NN will be biased toward high-magnitude features!

---

## Problem 3: Handling Missing Values in K-NN

### Challenge:
Test point has missing features: e.g., (5, ?, 3) where feature 2 is missing.

### Solution: Modified Distance Metric

**Approach:** Only compute distance over available features and normalize.

**Standard Euclidean Distance:**
```
d(x, y) = √[Σᵢ (xᵢ - yᵢ)²]
```

**Modified Distance (with missing values):**
```
d_modified(x, y) = √[n/n_available × Σᵢ∈available (xᵢ - yᵢ)²]
```

Where:
- n = total features
- n_available = non-missing features
- Scaling factor (n/n_available) normalizes for dimensionality

**Example:**
- Training: (2, 4, 3)
- Test: (5, ?, 3) — Feature 2 missing
- Distance: √[3/2 × ((5-2)² + (3-3)²)] = √[1.5 × 9] = 3.67

**Why this works:** 
- Simple to implement
- No data imputation needed
- Uses only reliable information
- Normalizes for reduced dimensionality

---

## Problem 4: K-NN Success in High-Dimensional Image Data

### Apparent Paradox:
Images have thousands of pixels (dimensions), but K-NN still works well. Why doesn't the "curse of dimensionality" destroy performance?

---

### Reason 1: **Intrinsic Dimensionality is Low**

**Explanation:**  
Although images nominally have d=10,000+ dimensions (pixels), the **effective dimensionality is much lower**.

**Why:**
- Natural images lie on a **low-dimensional manifold** embedded in high-dimensional space
- Pixels are highly correlated (e.g., neighboring pixels similar)
- Images have structure: edges, textures, objects
- Most of the 10,000 dimensions are redundant

**Analogy:**  
A 2D sheet of paper (low intrinsic dimension) can exist in 3D space. Images are like a low-D manifold in high-D pixel space.

**Evidence:**
- PCA on images: 95% variance captured by ~100 components (out of 10,000)
- This means effective dimension ≈ 100, not 10,000

---

### Reason 2: **Distance Metrics Still Meaningful**

**The Curse of Dimensionality predicts:**
In high dimensions, all distances become similar → nearest neighbors aren't actually "near."

**Why this doesn't happen for images:**

1. **Structured Data:** Image pixels aren't random; they have spatial correlation
2. **Relevant Dimensions Dominate:** Discriminative features (edges, shapes) vary more than noise
3. **L2 Distance Works:** Euclidean distance still captures perceptual similarity for images

**Example:**
- Two images of dogs: differ mainly in pose/lighting (few dimensions)
- Dog vs. car: differ in fundamental structure (many dimensions)
- Distance ratio: similar dogs are MUCH closer than dog-car

---

### Reason 3: **Large Training Sets**

**Explanation:**  
K-NN requires dense coverage of the feature space. Image datasets often have:
- ImageNet: 1.2M images
- MNIST: 60K images

**Why this helps:**
- Even in high-D space, with enough data, neighborhoods are well-populated
- Rule of thumb: need N ≈ e^d samples for curse to dominate
- But if intrinsic dimension is low (d_eff ≈ 100), N=60,000 is sufficient

---

### Reason 4: **Locality-Sensitive Hashing (LSH) and Approximate K-NN**

**Modern K-NN systems use:**
- Approximate nearest neighbor search (e.g., LSH, Annoy, FAISS)
- These methods exploit low intrinsic dimensionality
- Trade exact search for speed without significant accuracy loss

---

### Reason 5: **Feature Engineering / Learned Representations**

**In practice, K-NN on images often uses:**
- Pre-extracted features (HOG, SIFT)
- Deep learning embeddings (CNN features)

**Example:**
- Raw pixels: 28×28 = 784 dimensions
- CNN embedding: 128 dimensions
- Effective dimension after CNN: ~20-50

K-NN in embedding space works even better!

---

### Mathematical Insight:

**Curse of Dimensionality predicts:**
```
Distance concentration: max_dist / min_dist → 1 as d → ∞
```

**But for real images:**
```
max_dist / min_dist ≈ 2-5 (still distinguishable!)
```

Because intrinsic dimension d_eff << d_nominal.

---

### Empirical Evidence:

**MNIST (28×28 = 784 dimensions):**
- 1-NN achieves ~97% accuracy
- If curse of dimensionality applied, would get ~50% (random)

**CIFAR-10 (32×32×3 = 3,072 dimensions):**
- K-NN with proper distance metric: ~35-40% accuracy
- With learned features: ~80% accuracy

---

### Summary Answer:

**K-NN works well on high-dimensional images because:**

1. **Low intrinsic dimensionality** — Images lie on low-D manifolds
2.  **Structured correlations** — Pixels aren't independent random variables
3. **Meaningful distances** — Perceptual similarity preserved by L2 distance
4. **Large datasets** — Modern datasets provide dense coverage
5. **Feature engineering** — CNNs/HOG reduce effective dimensionality further

**Key Takeaway:**  
The curse of dimensionality assumes uniformly distributed data in all dimensions. Real-world images violate this assumption spectacularly — they're highly structured, correlated, and low-dimensional in their essence.

---

## Additional Notes:

**When K-NN FAILS on images:**
- Very few training examples per class
- High noise/occlusion
- Need rotation/scale invariance (raw pixels aren't invariant)

**When to use K-NN for images:**
- Quick baseline
- Small datasets where deep learning overfits
- With good feature extraction (HOG, SIFT, CNN embeddings)
- Non-parametric modeling needed

**Best Practice:**
- Always use feature extraction (don't use raw pixels)
- Scale features to [0,1] or standardize
- Use approximate K-NN (FAISS) for large datasets
- Try different distance metrics (L1, L2, cosine)

### Problem 3: Part 1

You are given a fully trained Perceptron model with weight vector **w**, along with training set **D_TR** and test set **D_TE**.

1. Your co-worker suggests evaluating $h(x) = sign(w \cdot x)$ for every $(x, y)$ in D_TR and D_TE. Does this help determine whether test error is higher than training error?
2. Why is there no need to compute training error explicitly for the Perceptron algorithm?

In [10]:
 1: NO - Just computing h(x) on training and test sets doesn't help determine if test error is meaningfully higher. 
You need statistical validation (confidence intervals, cross-validation, or hypothesis testing) to know if the
difference is real or just random variation.


2: Training error doesn't need explicit computation because the Perceptron convergence theorem 
guarantees zero training error when data is linearly separable. The algorithm only stops when
all training points are correctly classified, so training error = 0% by construction.

### Problem 3: Two-point 2D Dataset (Part 2)

Run the Perceptron algorithm **by hand or in code** on the following data:

1. Positive class: (10, -2)
2. Negative class: (12, 2)

Start with $w_0 = (0, 0)$ and a learning rate of 1.

- Compute how many updates are required until convergence.
- Write down the sequence of $w_i$ vectors.

In [13]:
# Iteration 1
Starting with w₀ = (0, 0)
Check x₁ = (10, -2), y₁ = +1:

w₀ · x₁ = (0)(10) + (0)(-2) = 0
y₁ · (w₀ · x₁) = (+1)(0) = 0 ≤ 0 ✗ Misclassified!

Update:

w₁ = w₀ + η · y₁ · x₁
w₁ = (0, 0) + 1 · (+1) · (10, -2)
w₁ = (10, -2)


Iteration 2
Check x₂ = (12, 2), y₂ = -1:

w₁ · x₂ = (10)(12) + (-2)(2) = 120 - 4 = 116
y₂ · (w₁ · x₂) = (-1)(116) = -116 ≤ 0 ✗ Misclassified!

Update:

w₂ = w₁ + η · y₂ · x₂
w₂ = (10, -2) + 1 · (-1) · (12, 2)
w₂ = (10, -2) + (-12, -2)
w₂ = (-2, -4)


Iteration 3
Check x₁ = (10, -2), y₁ = +1:

w₂ · x₁ = (-2)(10) + (-4)(-2) = -20 + 8 = -12
y₁ · (w₂ · x₁) = (+1)(-12) = -12 ≤ 0 ✗ Misclassified!

Update:

w₃ = w₂ + η · y₁ · x₁
w₃ = (-2, -4) + 1 · (+1) · (10, -2)
w₃ = (-2, -4) + (10, -2)
w₃ = (8, -6)


Iteration 4
Check x₂ = (12, 2), y₂ = -1:

w₃ · x₂ = (8)(12) + (-6)(2) = 96 - 12 = 84
y₂ · (w₃ · x₂) = (-1)(84) = -84 ≤ 0 ✗ Misclassified!

Update:

w₄ = w₃ + η · y₂ · x₂
w₄ = (8, -6) + 1 · (-1) · (12, 2)
w₄ = (8, -6) + (-12, -2)
w₄ = (-4, -8)


Iteration 5
Check x₁ = (10, -2), y₁ = +1:

w₄ · x₁ = (-4)(10) + (-8)(-2) = -40 + 16 = -24
y₁ · (w₄ · x₁) = (+1)(-24) = -24 ≤ 0 ✗ Misclassified!

Update:

w₅ = w₄ + η · y₁ · x₁
w₅ = (-4, -8) + 1 · (+1) · (10, -2)
w₅ = (-4, -8) + (10, -2)
w₅ = (6, -10)


Iteration 6
Check x₂ = (12, 2), y₂ = -1:

w₅ · x₂ = (6)(12) + (-10)(2) = 72 - 20 = 52
y₂ · (w₅ · x₂) = (-1)(52) = -52 ≤ 0 ✗ Misclassified!

Update:

w₆ = w₅ + η · y₂ · x₂
w₆ = (6, -10) + 1 · (-1) · (12, 2)
w₆ = (6, -10) + (-12, -2)
w₆ = (-6, -12)


Iteration 7
Check x₁ = (10, -2), y₁ = +1:

w₆ · x₁ = (-6)(10) + (-12)(-2) = -60 + 24 = -36
y₁ · (w₆ · x₁) = (+1)(-36) = -36 ≤ 0 ✗ Misclassified!

Update:

w₇ = w₆ + η · y₁ · x₁
w₇ = (-6, -12) + 1 · (+1) · (10, -2)
w₇ = (4, -14)


Final Check
Check x₂ = (12, 2), y₂ = -1:

w₇ · x₂ = (4)(12) + (-14)(2) = 48 - 28 = 20
y₂ · (w₇ · x₂) = (-1)(20) = -20 ≤ 0 ✗ Still misclassified!


Conclusion
The pattern continues indefinitely - the Perceptron algorithm does NOT 
converge on this dataset. This is because the two points are not linearly separable
(they cannot be separated by a straight line given their labels).
#

### Problem 4: Reconstructing the Weight Vector

Given the log of Perceptron updates:

| x | y | count |
|---|---|--------|
| (0, 0, 0, 0, 4) | +1 | 2 |
| (0, 0, 6, 5, 0) | +1 | 1 |
| (3, 0, 0, 0, 0) | -1 | 1 |
| (0, 9, 3, 6, 0) | -1 | 1 |
| (0, 1, 0, 2, 5) | -1 | 1 |

Assume learning rate = 1 and initial weight $w_0 = (0, 0, 0, 0, 0)$.

Compute the final weight vector after all updates.

In [16]:
#

Update 1: x = (0, 0, 0, 0, 4), y = +1, count = 2

Apply twice: w = (0, 0, 0, 0, 0) + 2(+1)(0, 0, 0, 0, 4)
w₁ = (0, 0, 0, 0, 8)

Update 2: x = (0, 0, 6, 5, 0), y = +1, count = 1

w₂ = (0, 0, 0, 0, 8) + (+1)(0, 0, 6, 5, 0)
w₂ = (0, 0, 6, 5, 8)

Update 3: x = (3, 0, 0, 0, 0), y = -1, count = 1

w₃ = (0, 0, 6, 5, 8) + (-1)(3, 0, 0, 0, 0)
w₃ = (-3, 0, 6, 5, 8)

Update 4: x = (0, 9, 3, 6, 0), y = -1, count = 1

w₄ = (-3, 0, 6, 5, 8) + (-1)(0, 9, 3, 6, 0)
w₄ = (-3, -9, 3, -1, 8)

Update 5: x = (0, 1, 0, 2, 5), y = -1, count = 1

w₅ = (-3, -9, 3, -1, 8) + (-1)(0, 1, 0, 2, 5)
w₅ = (-3, -10, 3, -3, 3)


Final weight vector: w = (-3, -10, 3, -3, 3)
#

### Problem 5: Visualizing Perceptron Convergence

Implement a Perceptron on a small 2D dataset with positive and negative examples.

- Plot the data points.
- After each update, visualize the decision boundary.
- Show how it converges to a stable separator.

In [19]:
#Todo
pythonimport numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib.patches as patches

# Dataset: linearly separable points
positive_points = np.array([[3, 4], [4, 5], [2, 3], [3, 2], [4, 3]])
negative_points = np.array([[1, 1], [2, 1], [1, 2], [0, 1], [1, 0]])

class Perceptron:
    def __init__(self, learning_rate=1):
        self.weights = np.array([0.0, 0.0, 0.0])  # [bias, w1, w2]
        self.lr = learning_rate
        self.history = []
        self.iteration = 0
        
    def predict(self, x):
        activation = self.weights[0] + np.dot(self.weights[1:], x)
        return 1 if activation >= 0 else -1
    
    def train_step(self, x, y):
        prediction = self.predict(x)
        if prediction != y:
            self.weights[0] += self.lr * y
            self.weights[1:] += self.lr * y * x
            return True
        return False
    
    def get_boundary(self):
        if self.weights[2] == 0:
            return None, None
        x = np.linspace(-1, 6, 100)
        y = -(self.weights[0] + self.weights[1] * x) / self.weights[2]
        return x, y

# Initialize perceptron
perceptron = Perceptron()

# Prepare all points for training
all_points = [(p, 1) for p in positive_points] + [(p, -1) for p in negative_points]

# Store history for animation
states = [{'weights': perceptron.weights.copy(), 'iteration': 0, 'current_point': None}]

# Train and record each step
max_epochs = 50
for epoch in range(max_epochs):
    converged = True
    for point, label in all_points:
        perceptron.iteration += 1
        if perceptron.train_step(point, label):
            converged = False
            states.append({
                'weights': perceptron.weights.copy(),
                'iteration': perceptron.iteration,
                'current_point': point,
                'label': label
            })
    if converged:
        print(f"✓ Converged after {epoch + 1} epochs and {perceptron.iteration} iterations")
        break

print(f"Final weights: w0={perceptron.weights[0]:.2f}, w1={perceptron.weights[1]:.2f}, w2={perceptron.weights[2]:.2f}")
print(f"Total updates: {len(states) - 1}")
print(f"Decision boundary: {perceptron.weights[0]:.2f} + {perceptron.weights[1]:.2f}*x + {perceptron.weights[2]:.2f}*y = 0")

# Create animation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

def init():
    ax1.clear()
    ax2.clear()
    return []

def animate(frame):
    ax1.clear()
    ax2.clear()
    
    state = states[frame]
    weights = state['weights']
    current_point = state.get('current_point')
    
    # Left plot: Main visualization
    ax1.set_xlim(-0.5, 5.5)
    ax1.set_ylim(-0.5, 5.5)
    ax1.set_xlabel('x₁', fontsize=12)
    ax1.set_ylabel('x₂', fontsize=12)
    ax1.set_title(f'Iteration {state["iteration"]} | Weights: [{weights[0]:.2f}, {weights[1]:.2f}, {weights[2]:.2f}]', 
                  fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.set_aspect('equal')
    
    # Plot positive points
    for point in positive_points:
        color = 'gold' if current_point is not None and np.array_equal(point, current_point) else 'red'
        edgecolor = 'orange' if current_point is not None and np.array_equal(point, current_point) else 'darkred'
        linewidth = 4 if current_point is not None and np.array_equal(point, current_point) else 2
        ax1.scatter(point[0], point[1], c=color, s=200, edgecolors=edgecolor, 
                   linewidth=linewidth, zorder=3, label='Positive (+1)' if point[0] == positive_points[0][0] else '')
    
    # Plot negative points
    for point in negative_points:
        color = 'gold' if current_point is not None and np.array_equal(point, current_point) else 'blue'
        edgecolor = 'orange' if current_point is not None and np.array_equal(point, current_point) else 'darkblue'
        linewidth = 4 if current_point is not None and np.array_equal(point, current_point) else 2
        ax1.scatter(point[0], point[1], c=color, s=200, edgecolors=edgecolor, 
                   linewidth=linewidth, zorder=3, label='Negative (-1)' if point[0] == negative_points[0][0] else '')
    
    # Plot decision boundary
    if weights[2] != 0:
        x_line = np.linspace(-1, 6, 100)
        y_line = -(weights[0] + weights[1] * x_line) / weights[2]
        is_converged = frame == len(states) - 1
        ax1.plot(x_line, y_line, 
                'g-' if is_converged else 'b--', 
                linewidth=3, 
                label='Decision Boundary (Converged)' if is_converged else 'Decision Boundary',
                zorder=2)
    
    ax1.legend(loc='upper right', fontsize=10)
    
    # Right plot: Weight evolution
    ax2.set_xlabel('Iteration', fontsize=12)
    ax2.set_ylabel('Weight Value', fontsize=12)
    ax2.set_title('Weight Evolution Over Time', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    iterations = [s['iteration'] for s in states[:frame+1]]
    w0_vals = [s['weights'][0] for s in states[:frame+1]]
    w1_vals = [s['weights'][1] for s in states[:frame+1]]
    w2_vals = [s['weights'][2] for s in states[:frame+1]]
    
    ax2.plot(iterations, w0_vals, 'ro-', label='w₀ (bias)', linewidth=2, markersize=6)
    ax2.plot(iterations, w1_vals, 'go-', label='w₁', linewidth=2, markersize=6)
    ax2.plot(iterations, w2_vals, 'bo-', label='w₂', linewidth=2, markersize=6)
    ax2.legend(loc='best', fontsize=10)
    
    # Add convergence marker
    if frame == len(states) - 1:
        ax2.axvline(x=state['iteration'], color='green', linestyle='--', linewidth=2, alpha=0.5)
        ax2.text(state['iteration'], ax2.get_ylim()[1] * 0.9, 'Converged', 
                rotation=90, verticalalignment='top', fontsize=10, color='green', fontweight='bold')
    
    return []

# Create animation
anim = FuncAnimation(fig, animate, init_func=init, frames=len(states), 
                    interval=500, blit=True, repeat=True)

plt.tight_layout()
plt.close()

# Display animation
HTML(anim.to_jshtml())